In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="quick-starts-ws-165838")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-167112
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-167112


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':uniform(0.1,1),
'--max_iter':quniform(100,1500,100)}

)

# Specify a Policy
policy = early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

script_folder='./training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)

env=Environment.get(workspace=ws, name="AzureML-Tutorial")

src= ScriptRunConfig(source_directory= script_folder,
                    script='train.py',
                    compute_target= aml_compute,
                     environment=env
                    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy", 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=15,
                             max_concurrent_runs=3)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr=exp.submit(config=hyperdrive_config)

In [5]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
import joblib
# Get your best run and save the model from that run.
best_run=hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments']) 
print(best_run.get_file_names())

model=best_run.register_model(model_name="Best", model_path=".")

best_run_metrics= best_run.get_metrics()
print('\n Accuracy: ', best_run_metrics['Accuracy'])

### YOUR CODE HERE ###

['--C', '0.4461476409241234', '--max_iter', '300']
['logs/azureml/18_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']

 Accuracy:  0.909711684370258


In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [18]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset
import os 

if "training" not in os.listdir():
    os.mkdir("./training")

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

y_df=pd.DataFrame(y, columns=['y'])
x_df=pd.concat([x,y_df], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=66)

X_train.to_csv("training/train_data.csv", index=False)

data_store=ws.get_default_datastore()
data_store.upload(src_dir="./training", target_path='quick-starts-ws-165838', overwrite=True, show_progress=True)
train_data=Dataset.Tabular.from_delimited_files(path=data_store.path("quick-starts-ws-165838/train_data.csv"))


2021-12-20:19:14:33,925 INFO     [datastore_client.py:908] <azureml.core.authentication.InteractiveLoginAuthentication object at 0x7f957b28bda0>
2021-12-20:19:14:34,508 WARNING  [_dataset_deprecation.py:89] "Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.
2021-12-20:19:14:34,508 INFO     [azure_storage_datastore.py:754] Called AzureBlobDatastore.upload
2021-12-20:19:14:34,533 INFO     [azure_storage_datastore.py:352] Uploading an estimated of 4 files
2021-12-20:19:14:34,787 INFO     [azure_storage_datastore.py:352] Uploading ./training/.amlignore
2021-12-20:19:14:34,787 INFO     [azure_storage_datastore.py:352] Uploaded ./training/.amlignore, 1 files out of an estimated total of 4
2021-12-20:19:14:34,788 INFO     [azure_storage_datastore.py:352] Uploading ./training/.am

Uploading an estimated of 4 files
Uploading ./training/.amlignore
Uploaded ./training/.amlignore, 1 files out of an estimated total of 4
Uploading ./training/.amlignore.amltmp
Uploaded ./training/.amlignore.amltmp, 2 files out of an estimated total of 4
Uploading ./training/train.py
Uploaded ./training/train.py, 3 files out of an estimated total of 4
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 4 files out of an estimated total of 4
Uploaded 4 files


In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=aml_compute 
    )

In [23]:
# Submit your automl run

### YOUR CODE HERE ###
auto_exp=Experiment(workspace=ws, name="auto_exp")
automl_run=exp.submit(automl_config, show_output=False)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

Submitting remote run.


ValidationException: ValidationException:
	Message: Failed to execute the requested operation: data/settings validation. Error details: Validation error(s): [{
    "additional_properties": {
        "debugInfo": null
    },
    "code": "UserError",
    "severity": 2,
    "message": "Y is empty",
    "message_format": "{dataName} is empty",
    "message_parameters": {
        "dataName": "Y"
    },
    "reference_code": null,
    "details_uri": null,
    "target": "training_data",
    "details": [
        {
            "additional_properties": {
                "debugInfo": null
            },
            "code": null,
            "severity": null,
            "message": "null",
            "message_format": null,
            "message_parameters": {},
            "reference_code": null,
            "details_uri": null,
            "target": null,
            "details": [],
            "inner_error": null,
            "additional_info": null
        }
    ],
    "inner_error": {
        

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###